In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1422895,2021-07-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1422896,2021-07-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1422897,2021-07-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1422898,2021-07-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
37579,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37581,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37583,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37585,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37587,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
871487,2021-07-01,Arkansas,Arkansas,5001,2228,38.00,5000,Arkansas,AR,Arkansas,State,3017804
871489,2021-07-02,Arkansas,Arkansas,5001,2235,39.00,5000,Arkansas,AR,Arkansas,State,3017804
871491,2021-07-03,Arkansas,Arkansas,5001,2235,39.00,5000,Arkansas,AR,Arkansas,State,3017804
871493,2021-07-04,Arkansas,Arkansas,5001,2235,39.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1422895,2021-07-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1422896,2021-07-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1422897,2021-07-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1422898,2021-07-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-07-05') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
531,2021-07-05,Snohomish,Washington,40330,605.00,53061,WA,County,822083,4905.83,73.59
1060,2021-07-05,Cook,Illinois,557173,11049.00,17031,IL,County,5150233,10818.40,214.53
1588,2021-07-05,Orange,California,273738,5126.00,6059,CA,County,3175692,8619.79,161.41
2115,2021-07-05,Maricopa,Arizona,561596,10309.00,4013,AZ,County,4485414,12520.49,229.83
2642,2021-07-05,Los Angeles,California,1253404,24506.00,6037,CA,County,10039107,12485.21,244.11
...,...,...,...,...,...,...,...,...,...,...,...
1421959,2021-07-05,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1422225,2021-07-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1422460,2021-07-05,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1422691,2021-07-05,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
531,2021-07-05,Snohomish,Washington,40330,605.00,53061,WA,County,822083,4905.83,73.59,73.59,4905.83
1060,2021-07-05,Cook,Illinois,557173,11049.00,17031,IL,County,5150233,10818.40,214.53,214.53,10818.40
1588,2021-07-05,Orange,California,273738,5126.00,6059,CA,County,3175692,8619.79,161.41,161.41,8619.79
2115,2021-07-05,Maricopa,Arizona,561596,10309.00,4013,AZ,County,4485414,12520.49,229.83,229.83,12520.49
2642,2021-07-05,Los Angeles,California,1253404,24506.00,6037,CA,County,10039107,12485.21,244.11,244.11,12485.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421959,2021-07-05,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1422225,2021-07-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1422460,2021-07-05,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1422691,2021-07-05,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
911396,2021-07-05,Hale,Alabama,2273,77.00,1065,AL,County,14651,15514.30,525.56,525.56,15514.30,1
992341,2021-07-05,Clarke,Alabama,3531,61.00,1025,AL,County,23622,14947.93,258.23,258.23,14947.93,2
712581,2021-07-05,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3
631623,2021-07-05,Etowah,Alabama,14249,366.00,1055,AL,County,102268,13933.00,357.88,357.88,13933.00,4
529776,2021-07-05,Franklin,Alabama,4328,81.00,1059,AL,County,31362,13800.14,258.27,258.27,13800.14,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870103,2021-07-05,Sublette,Wyoming,797,7.00,56035,WY,County,9831,8107.01,71.20,71.20,8107.01,19
869638,2021-07-05,Converse,Wyoming,1038,18.00,56009,WY,County,13822,7509.77,130.23,130.23,7509.77,20
1082043,2021-07-05,Lincoln,Wyoming,1432,14.00,56023,WY,County,19830,7221.38,70.60,70.60,7221.38,21
1204979,2021-07-05,Niobrara,Wyoming,163,2.00,56027,WY,County,2356,6918.51,84.89,84.89,6918.51,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
712581,2021-07-05,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3,1
911396,2021-07-05,Hale,Alabama,2273,77.00,1065,AL,County,14651,15514.30,525.56,525.56,15514.30,1,2
629751,2021-07-05,Clay,Alabama,1603,59.00,1027,AL,County,13235,12111.82,445.79,445.79,12111.82,20,3
302464,2021-07-05,Walker,Alabama,7372,282.00,1127,AL,County,63521,11605.61,443.95,443.95,11605.61,30,4
768618,2021-07-05,Greene,Alabama,938,35.00,1063,AL,County,8111,11564.54,431.51,431.51,11564.54,31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082043,2021-07-05,Lincoln,Wyoming,1432,14.00,56023,WY,County,19830,7221.38,70.60,70.60,7221.38,21,19
766753,2021-07-05,Hot Springs,Wyoming,373,3.00,56017,WY,County,4413,8452.30,67.98,67.98,8452.30,17,20
1054962,2021-07-05,Uinta,Wyoming,2321,13.00,56041,WY,County,20226,11475.33,64.27,64.27,11475.33,5,21
301042,2021-07-05,Teton,Wyoming,3802,11.00,56039,WY,County,23464,16203.55,46.88,46.88,16203.55,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4905.83,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4905.83,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4905.83,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4905.83,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,73.59,4905.83,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419577,2021-07-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1419578,2021-07-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1419579,2021-07-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1419580,2021-07-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
907616,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15514.30,2,1,1.00
907617,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15514.30,2,1,0.00
907618,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15514.30,2,1,0.00
907619,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15514.30,2,1,0.00
907620,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,525.56,15514.30,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100247,2021-07-01,Crook,Wyoming,459,13.00,56011,WY,County,7584,6052.22,171.41,171.41,6052.22,6,23,0.00
1100248,2021-07-02,Crook,Wyoming,459,13.00,56011,WY,County,7584,6052.22,171.41,171.41,6052.22,6,23,0.00
1100249,2021-07-03,Crook,Wyoming,459,13.00,56011,WY,County,7584,6052.22,171.41,171.41,6052.22,6,23,0.00
1100250,2021-07-04,Crook,Wyoming,459,13.00,56011,WY,County,7584,6052.22,171.41,171.41,6052.22,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
709727,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,1.00,0.00
709728,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
709729,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
709730,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
709731,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707387,2021-07-01,Albany,Wyoming,4645,17.00,56001,WY,County,38880,11947.02,43.72,43.72,11965.02,23,3,8.00,0.00
707388,2021-07-02,Albany,Wyoming,4652,17.00,56001,WY,County,38880,11965.02,43.72,43.72,11965.02,23,3,7.00,0.00
707389,2021-07-03,Albany,Wyoming,4652,17.00,56001,WY,County,38880,11965.02,43.72,43.72,11965.02,23,3,0.00,0.00
707390,2021-07-04,Albany,Wyoming,4652,17.00,56001,WY,County,38880,11965.02,43.72,43.72,11965.02,23,3,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-07-05') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
361208,2021-07-05,Imperial,California,28981,741.00,6025,CA,County,181215,15992.61,408.91,408.91,15992.61,1,2,0.00,0.00
2642,2021-07-05,Los Angeles,California,1253404,24506.00,6037,CA,County,10039107,12485.21,244.11,244.11,12485.21,2,5,436.00,0.00
179598,2021-07-05,San Bernardino,California,299948,5003.00,6071,CA,County,2180085,13758.55,229.49,229.49,13758.55,3,4,0.00,0.00
633447,2021-07-05,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
80185,2021-07-05,Stanislaus,California,63792,1077.00,6099,CA,County,550660,11584.64,195.58,195.58,11584.64,5,9,0.00,0.00
63765,2021-07-05,San Joaquin,California,74900,1444.00,6077,CA,County,762148,9827.49,189.46,189.46,9827.49,6,17,0.00,0.00
37092,2021-07-05,Riverside,California,301820,4640.00,6065,CA,County,2470546,12216.73,187.81,187.81,12216.73,7,7,0.00,0.00
99462,2021-07-05,Tulare,California,49896,853.00,6107,CA,County,466195,10702.82,182.97,182.97,10702.82,8,10,0.00,0.00
573901,2021-07-05,Merced,California,32391,482.00,6047,CA,County,277680,11664.87,173.58,173.58,11664.87,9,8,0.00,0.00
36120,2021-07-05,Fresno,California,102512,1727.00,6019,CA,County,999101,10260.42,172.86,172.86,10260.42,10,14,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1345309,2021-07-05,Lassen,California,5822,26.00,6035,CA,County,30573,19042.95,85.04,85.04,19042.95,35,1,0.00,0.00
361208,2021-07-05,Imperial,California,28981,741.00,6025,CA,County,181215,15992.61,408.91,408.91,15992.61,1,2,0.00,0.00
768560,2021-07-05,Kings,California,23121,247.00,6031,CA,County,152940,15117.69,161.50,161.50,15117.69,11,3,2.00,0.00
179598,2021-07-05,San Bernardino,California,299948,5003.00,6071,CA,County,2180085,13758.55,229.49,229.49,13758.55,3,4,0.00,0.00
2642,2021-07-05,Los Angeles,California,1253404,24506.00,6037,CA,County,10039107,12485.21,244.11,244.11,12485.21,2,5,436.00,0.00
225909,2021-07-05,Kern,California,111111,1408.00,6029,CA,County,900202,12342.90,156.41,156.41,12342.90,13,6,0.00,0.00
37092,2021-07-05,Riverside,California,301820,4640.00,6065,CA,County,2470546,12216.73,187.81,187.81,12216.73,7,7,0.00,0.00
573901,2021-07-05,Merced,California,32391,482.00,6047,CA,County,277680,11664.87,173.58,173.58,11664.87,9,8,0.00,0.00
80185,2021-07-05,Stanislaus,California,63792,1077.00,6099,CA,County,550660,11584.64,195.58,195.58,11584.64,5,9,0.00,0.00
99462,2021-07-05,Tulare,California,49896,853.00,6107,CA,County,466195,10702.82,182.97,182.97,10702.82,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
361208,2021-07-05,Imperial,California,28981,741.00,6025,CA,County,181215,15992.61,408.91,408.91,15992.61,1,2,0.00,0.00
2642,2021-07-05,Los Angeles,California,1253404,24506.00,6037,CA,County,10039107,12485.21,244.11,244.11,12485.21,2,5,436.00,0.00
179598,2021-07-05,San Bernardino,California,299948,5003.00,6071,CA,County,2180085,13758.55,229.49,229.49,13758.55,3,4,0.00,0.00
633447,2021-07-05,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
80185,2021-07-05,Stanislaus,California,63792,1077.00,6099,CA,County,550660,11584.64,195.58,195.58,11584.64,5,9,0.00,0.00
63765,2021-07-05,San Joaquin,California,74900,1444.00,6077,CA,County,762148,9827.49,189.46,189.46,9827.49,6,17,0.00,0.00
37092,2021-07-05,Riverside,California,301820,4640.00,6065,CA,County,2470546,12216.73,187.81,187.81,12216.73,7,7,0.00,0.00
99462,2021-07-05,Tulare,California,49896,853.00,6107,CA,County,466195,10702.82,182.97,182.97,10702.82,8,10,0.00,0.00
573901,2021-07-05,Merced,California,32391,482.00,6047,CA,County,277680,11664.87,173.58,173.58,11664.87,9,8,0.00,0.00
36120,2021-07-05,Fresno,California,102512,1727.00,6019,CA,County,999101,10260.42,172.86,172.86,10260.42,10,14,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4742,07/01/21,Lassen,5821,26.00,19039.68,85.04,35,1,0.00,0.00
4743,07/02/21,Lassen,5822,26.00,19042.95,85.04,35,1,1.00,0.00
4744,07/03/21,Lassen,5822,26.00,19042.95,85.04,35,1,0.00,0.00
4745,07/04/21,Lassen,5822,26.00,19042.95,85.04,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4828,07/01/21,Fresno,102490,1727.00,10258.22,172.86,10,14,26.00,2.00
4829,07/02/21,Fresno,102512,1727.00,10260.42,172.86,10,14,22.00,0.00
4830,07/03/21,Fresno,102512,1727.00,10260.42,172.86,10,14,0.00,0.00
4831,07/04/21,Fresno,102512,1727.00,10260.42,172.86,10,14,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)